<a href="https://colab.research.google.com/github/kimdonggyu2008/2024_2_Capstone/blob/main/translator_train_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers datasets accelerate

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.2 MB/s eta 0:00:00


#transformer huggingface에서 로드하는 경우

In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("KETI-AIR/long-ke-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("KETI-AIR/long-ke-t5-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.49k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.17M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/891 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

#로컬에서 로딩하는 경우

In [5]:
import torch
import pandas as pd
from datasets import load_dataset
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments

In [6]:
# # 1. 토크나이저 로드
# tokenizer = AutoTokenizer.from_pretrained("path/to/tokenizer")

# # 2. 모델 로드
# model = AutoModelForSeq2SeqLM.from_pretrained("path/to/model", local_files_only=True)

#학습데이터 로드 및 토크나이징

In [7]:

# # 3. 데이터셋 로드 (예시 데이터셋)
# df = pd.read_csv('/content/drive/MyDrive/translator/data/train.csv')
# dataset = Dataset.from_pandas(df)


In [8]:
# import pickle
# from datasets import Dataset, DatasetDict

# # combined_df를 Dataset으로 변환
# dataset = Dataset.from_pandas(df)

# # train, test 데이터셋 분리 (예시로 10%를 테스트 세트로 분리)
# train_test = dataset.train_test_split(test_size=0.1)
# datasets = DatasetDict({
#     'train': train_test['train'],
#     'test': train_test['test']
# })

# # 전처리 함수 (한국어 -> 영어 번역)
# def preprocess_function(examples):
#     inputs = examples['ko']  # 'ko' 열에서 한국어 텍스트를 가져옴
#     targets = examples['en']  # 'en' 열에서 영어 번역 텍스트를 가져옴
#     model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

#     # Pegasus 모델은 입력과 출력이 모두 시퀀스이므로 target도 tokenizing이 필요
#     with tokenizer.as_target_tokenizer():
#         labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")

#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs



In [9]:
# # 데이터셋을 전처리 및 토크나이징 (datasets가 이미 로드된 상태에서)
# tokenized_datasets = datasets.map(preprocess_function, batched=True)

# # pkl 파일로 저장
# output_file = "/content/drive/MyDrive/translator/preprocessed/processed_dataset.pkl"
# with open(output_file, 'wb') as f:
#     pickle.dump(tokenized_datasets, f)

# print(f"Tokenized dataset saved to {output_file}")

#학습(토크나이징 된 데이터 받음)


In [13]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments, DataCollatorForSeq2Seq
import pickle

In [14]:
def load_preprocessed_data(file_path):
    with open(file_path, 'rb') as f:
        tokenized_datasets = pickle.load(f)
    return tokenized_datasets

In [15]:
input_file = "/content/drive/MyDrive/translator/preprocessed/processed_dataset.pkl"
loaded_data = load_preprocessed_data(input_file)

In [16]:
train_data = loaded_data['train']
eval_data = loaded_data['test']

In [17]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [21]:

# 6. 학습 파라미터 설정 (중간에 멈춰도 저장되도록 설정)
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/translator/model",  # 모델 및 체크포인트가 저장될 디렉토리
    evaluation_strategy="steps",  # 매 에포크마다 평가
    save_strategy="steps",  # 스텝마다 체크포인트 저장
    save_steps=500,  # 500 스텝마다 체크포인트 저장
    save_total_limit=3,  # 최근 3개의 체크포인트만 유지
    logging_steps=100,  # 로그를 출력할 스텝 간격
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,  # 학습 종료 시 가장 성능이 좋은 모델 불러오기
    save_on_each_node=False,  # 모든 노드에서 체크포인트를 저장하지 않음 (분산 학습 시)
    logging_dir="/content/drive/MyDrive/translator/logging",  # 로그를 저장할 디렉토리
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [23]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,  # 평가 데이터셋 추가
    data_collator=data_collator,
)

In [24]:

# 8. 학습 시작
try:
    trainer.train()
except KeyboardInterrupt:
    print("학습이 중단되었습니다. 마지막 체크포인트가 저장되었습니다.")


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1080: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Step,Training Loss,Validation Loss


학습이 중단되었습니다. 마지막 체크포인트가 저장되었습니다.


In [ ]:

# 9. 최종 모델 저장
trainer.save_model("./results/final_model")


In [ ]:

# 10. 토크나이저 저장 (필요할 경우)
tokenizer.save_pretrained("./results/final_model")